### čtvrtá lekce

2. `pandas` II.,
    - file io, encoding, chybějící data,
    - kombinace datasetů, concat, append, merge, join,
    - časové řady, možná?
    - eval() a query() jako high-perf operace.

### třetí lekce

2. `pandas` I.,
    - Úvod do frameworku, proč je dobré jej ovládat, kde s ním pracovat?
    - jaké jsou základní datové typy? (Dframe, series, časové řady)
    - základní orientace, indexování,

## Druhá lekce, kaggle
* reading and writting DFs,
* indexing, selecting rows & columns, conditional selection,
* functions, map and lambdas,
* groupby, multiindex, sorting values,
* data types or Dtypes, convert data types,
* missing values, fillna, replace,
* rename, merge, join, concat.
* missing data points, the reason why are the data missing, drop missing values, filling missing vals,
* scaling: changing the range, normalization: changing shape,
* parsing dates, selecting days,
* encoding: what are encodings, encode, decode, identifying and saving examples,
* inconsistency: fuzzywuzzy

## Druhá lekce, kniha
* pandas objects: DFs, Series, Index,
* indexing and selection: series, DFs,
* operating: w/ numpy (hard to implement without numpy),
* handling missing data: Nan and None in pandas,detecting, droping, filling null values,
* indexing: single index, multiple indexes(bad and good example), indexing and slicing multiindex,
* concat, append, merge, join,
* agregation: count, first, last, mean, median, min, max,
* groupBy: split, apply, combine,
* pivot table, methods similar to python methods, regex finding,slice & other miscellaneous methods,
* Time series, to_datetime, data structures of time series, date_range, period_range, timedelta_range,
* frenquencies & offsets, resampling, time shifting,
* eval and query, comparing eval and without eval,


## Druhá lekce, Honza
* manipulace, úvod, instalace,
* DataFrame,
* Series,
* Načítání dat,
* selekce dat,
* indexování, loc a iloc,
* přejmenování, přidávání, odstraňování sloupců,
* Nan, null, nahrazování,
* concat, merge, drop_duplicates, sort_values,
* unique, apply,
* kopie a práce s kopiemi,
* statistika,
* čtení dat online,

<br>

## Časové řady

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.odqbr-09dxxBQjF7x7NyBAHaII%26pid%3DApi&f=1&ipt=5b4a03c1dd38b52108237102ebb350595a5d6ea6a50572f5897ca89fbf5b5445&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

Část *frameworku* byla vyvinuta za účelem **finančního modelování**.

Proto je více než dobře vybavena sadou nástrojů, které umí pracovat **s daty, časem a časovými objekty**.

<br>

Uplatnění:
* **Analýza a predikce**: Časové řady umožňují analyzovat vývoj hodnot v čase a předpovídat budoucí trendy nebo sezónní změny. To je klíčové v oblastech jako finance, ekonomika, prodej, počasí a mnoho dalších.

* **Efektivní manipulace s časem**: Pandas poskytuje nástroje pro efektivní práci s časovými řadami, což zjednodušuje úkoly, jako je agregace, interpolace nebo časové posuny. Díky integrovaným funkcím můžete snadno pracovat s daty různých časových frekvencí a konvertovat mezi nimi.

* **Časově závislé analýzy**: Časové řady umožňují provádět časově závislé analýzy, jako je detekce změn v trendech, identifikace sezónních vlivů nebo identifikace časových závislostí mezi proměnnými.

* **Flexibilní indexace**: Pandas podporuje flexibilní indexaci časových řad, což usnadňuje filtrování, řazení a výběr dat na základě časových značek nebo rozsahů.

* **Kompatibilita s dalšími knihovnami**: Pandas je kompatibilní s mnoha dalšími knihovnami pro analýzu časových řad, jako je statsmodels nebo scikit-learn. To usnadňuje integraci a rozšíření vašich analýz s pokročilejšími metodami a algoritmy.
    
<br>

Jde například o údaje typu:
* *timestampy*, údaj odkazující na konkrétní časový okamžik (např. `4. července 2015 v 7:00 hod.`),
* *časové intervaly*, tedy období odkazují na délku času mezi konkrétním začátkem a koncem (např. intervaly ze dne na den),
* *time delta* objekty, tedy přesné délky času (např. 22,22 sekundy).

#### Data a čas v Pythonu

---

Standardní výbavou Pythonu jsou knihovny `datetime`:

In [ ]:
from datetime import datetime

In [ ]:
datetime(year=2023, month=4, day=5)

<br>

Nebo knihovna `dateutil` pro parsování datových typů z různých stringových zadání:

In [ ]:
from dateutil import parser

In [ ]:
date = parser.parse("5th of april, 2023")

In [ ]:
date

Kde pomocí metody `strftime` můžeš vypsat den:

In [ ]:
date.strftime("%A")

#### Data a čas v numpy

---

Některé nedostatky uvnitř knihoven `datetime` a `dateutil` vedli ke vzniku sady nástrojů.

Tyto doplňky vznikly pod hlavičkou knihovny `numpy`.

In [ ]:
from numpy import array, arange

In [ ]:
date = array('2023-04-05', dtype=np.datetime64)

In [ ]:
date

<br>

Pokud potřebuješ pole následujících 7 dní:

In [ ]:
date + arange(7)

<br>

Vzhledem k jednotnému datovu typu v poli pro **numpy** `datetime64` může tento typ operace
provádět mnohem rychleji, než přímo v Pythonu `datetime` objekty, zejména když objekty nabývají na velikosti.

#### Data a čas v pandách

---

Jde o kombinace objektů z obou předchozích podkapitol.

Ty dávají dohromady to nejlepší prostředky pro zacházení s časem.

In [ ]:
from pandas import to_datetime

In [ ]:
date = to_datetime("5th of April, 2023")

In [ ]:
date

In [ ]:
date.strftime("%A")

<br>

### Časové řady

---

V podstatě jde o hlavní nástroj, který tato knihovna dovede nabídnout.

#### Indexovní časem
`DatetimeIndex` obsahuje časové značky (*timestamp*), které jsou uloženy ve formátu `datetime64` s nanosekundovou přesností.

Tento objekt umožňuje efektivní práci s časovými řadami a poskytuje mnoho funkcí pro manipulaci s daty a časy.

Výhody práce s `DatetimeIndex` objektem:
* Časové zóny,
* frekvence,
* časově závislé selekce,
* atributy časových značek,
* operace s časem.

In [ ]:
from pandas import DatetimeIndex, Series

In [ ]:
datumy = ["2023-04-05", "2022-04-05", "2021-04-05", "2020-04-05"]

In [ ]:
indexy = DatetimeIndex(datumy)

In [ ]:
hodnoty = [to_datetime(den).strftime("%A") for den in datumy]

In [ ]:
df_hodnoty = Series(hodnoty, index=indexy)

In [ ]:
df_hodnoty

In [ ]:
type(indexy)

In [ ]:
df_hodnoty["2020": "2022"]

<br>

### Základní objekty

---

Mezi základní objekty pro práci s časem patří:
* `Timestamp` typ (související struktura Indexu `DatetimeIndex`,
* `Period` typ (.. `PeriodIndex`),
* `Timedelta` typ (.. `TimedeltaIndex`).

#### Timestamp & DatetimeIndex

Nejčastější datové typy, které lze vyvolat přímo, ovšem běžnější je pracovat s funkcí `to_datetime`.

Funkce `to_datetime` umí parsovat různé stringové formáty.

In [ ]:
from pandas import to_datetime

#### Práce s jedním datumem

In [ ]:
datum = to_datetime("05/04/2023")

In [ ]:
datum

In [ ]:
type(datum)

Pokud do funkce `to_datetime` vložíš jedinou hodnotu, vrací objekty typu `Timestamp`.

<br>

#### Specifický formát

Pokud se parser ztratí nebo tvoje zadání neodpovídá jeho vyhotovení:

In [ ]:
pd.to_datetime("12-11-2010 00:00", format="%d-%m-%Y %H:%M")

<br>

#### Práce s několika datumy

In [ ]:
datumy = to_datetime([
    datetime(2023, 4, 5), "5th of April 2023", "2023-Apr-5", "05-04-2023", "20230405"]
)

In [ ]:
datumy

In [ ]:
type(datumy)

Zatímco pole hodnot s datumy, které funkce `to_datetime` zpracuje vrací objekt typu `DatetimeIndex`.

<br>

Objekt `DatetimeIndex` potom můžeš konvertovat na `PeriodIndex` pomocí *metody* `to_period`:

In [ ]:
datumy.to_period?

<br>

Tato metoda je užitečná, pokud chcete převést časové řady **na určité časové období**.

Když budeš třeba potřebovat převést hodnoty **z denních dat na měsíční data**.

#### Denní data

In [ ]:
datumy.to_period("D")

#### Měsíční data

In [ ]:
datumy.to_period("M")

In [ ]:
data = {
    'date': pd.date_range(start='2022-01-01', periods=10, freq='D'),
    'sales': [100, 102, 105, 107, 110, 112, 115, 117, 120, 122]
}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.set_index('date', inplace=True)

In [ ]:
df_period = df.to_period(freq='Y')

<br>

### TimedeltaIndex

---

Jde o rozdíl v jednotkách času.

Nejčastěji se s tímto objektem setkáš, pokud potřebuješ získat **rozdíl mezi dvěma datumy**.

In [ ]:
specialni_datumy = to_datetime([
    datetime(2022, 4, 5), "5th of April 2021", "2020-Apr-5", "05-04-2019", "20180405"]
)

In [ ]:
datumy - specialni_datumy

<br>

### Funkce `date_range`

---

Aby bylo zadání řady (sekvence) dat pohodlnější, vyzkoušej funkci `date_range`.

Obdobně potom pracují související funkce:
* `date_range`, timestampy,
* `period_range`, periody,
* `timedelta_range`, pro delty.

In [ ]:
from pandas import date_range

<br>

#### Počet period

In [ ]:
datumy_ind = date_range("01-01-1992", periods=8)

In [ ]:
datumy_ind

<br>

#### Frekvence

In [ ]:
datumy_mesicne_ind = date_range("01-01-1992", periods=8, freq="M")

In [ ]:
datumy_mesicne_ind

<br>

#### Hodinové periody

In [ ]:
from pandas import timedelta_range

In [ ]:
hodinove_ind = timedelta_range(0, periods=12, freq="H")

In [ ]:
hodinove_ind

<br>

### Frekvence

Časovou paletou frekvencí, kterou framework `pandas` nabízí je tato tabulka:

| String | Popisek |
| :-: | :- |
| `D` | kalendářní den |
| `W` | týden |
| `M` | konec měsíce |
| `Q` | konec čtvrtletí |
| `A` | konec roku |
| `H` | hodiny |
| `T` | minuty |
| `S` | vteřiny |
| `B` | pracovní den |
| `BM` | konec pracovníḧo měsíce |
| `BQ` | konec pracovního čtvrtletí |

#### Netradiční frekvence

In [ ]:
timedelta_range(0, periods=5, freq="1H15T")

### Resampling

---

*Resampling* nebo také *převzorkování* je proces, který upravuje frekvenci časové řady.

*Resampling* se obvykle používá **pro snížení frekvence** (z hodinových dat na denní data).

*Resampling* zahrnuje *agregaci* dat.

V `pandas` se k tomu používá metoda `resample`, která má jako parametr novou frekvenci, na kterou chcete data převést:

In [ ]:
import pandas as pd

In [ ]:
data = {
    'date': pd.date_range(start='2022-01-01', periods=60, freq='D'),
    'sales': range(60)
}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.set_index('date', inplace=True)

In [ ]:
monthly_data = df.resample('M').sum()

In [ ]:
print(monthly_data)

<br>

### Zpřesňování

---

Zahrnuje **zvýšení frekvence časových řad**.

Přičemž se nově vytvořené hodnoty obvykle interpolují nebo doplňují nějakou konstantou.

V pandas se k zpřesňování používá metoda `asfreq`, která má jako parametr novou frekvenci, na kterou chcete data převést.

In [ ]:
data = {
    'date': pd.date_range(start='2022-01-01', end='2022-03-01', freq='MS'),
    'sales': [100, 120, 150]
}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df.set_index('date', inplace=True)

In [ ]:
daily_data = df.asfreq('D', method='ffill')

In [ ]:
daily_data.head(20)

Metoda `asfreq` bere jako parametr novou frekvenci, na kterou chceme data převést.

V případě výše jde o frekvenci `D`, což znamená **denní data**.

Protože při zpřesňování se **vytvoří chybějící hodnoty** (v důsledku zvýšení frekvence), je potřeba zvolit metodu pro jejich doplnění.

Výš jde o metodu `forward fill` (parametr `method='ffill'`), která kopíruje předchozí hodnotu pro výplň chybějících hodnot.

Další možností je použít metodu `'back fill'` (parametr `method='bfill'`), která kopíruje následující hodnotu pro výplň chybějících hodnot.

<br>

**🧠 CVIČENÍ 🧠, procvič si časové řady**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Celkový prodej za každý měsíc.
2. Průměrný prodej za každý den v týdnu.
3. Denní prodej za poslední týden.

In [133]:
from numpy.random import randint
from pandas import date_range, DataFrame

denni_datumy = date_range(start='2022-01-01', end='2022-12-31', freq='D')
prodeje = randint(10, 100, size=(len(denni_datumy),))

data = {
    'datumy': denni_datumy,
    'prodeje': prodeje
}

df_prodeje = DataFrame(data)
df_prodeje.set_index('datumy', inplace=True)

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    mesicni_prodej = df_prodeje.resample('M').sum()

    df_prodeje['weekday'] = df_prodeje.index.weekday
    denni_prumer = df_prodeje.groupby('weekday')['prodeje'].mean()

    prodej_posledni_vikend = df_prodeje.loc['2022-12-25':, 'prodeje']
    ```
</details>

* [High performance](),
    - [úvodní motivace](),
    - [eval](),
    - [query](),
    - [caveats]().

## Vysoký výkon v PANDAS

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.riOR-y2_yu_2cGdEN2e6-wAAAA%26pid%3DApi&f=1&ipt=638c4516d155ff9768066e17887a621cbad0de6bce639a40be98f5dedbce67ac&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

Z předchozích ukázek a popisků (seskupování) tedy můžeš chápat, že skutečný výkon *frameworku* tkví v převedení **základních operací do jazyka C**.

Tyto abstrakce jsou účinné a efektivní pro mnoho běžných případů použití.

Vytvářejí ale spousty dočasných objektů, což může způsobit nepřiměřenou režii výpočetního času a paměti.

### EVAL a QUERY

---

Přímý přístup k operacím rychlosti C bez nákladné alokace `pandas` poskytuje funkce `eval` a `query`.

Ty spolehájí na funkci `numexpr`.

<br>

### funkce EVAL

---

Dva důvody, proč používat funkci `eval` místo obyčejného Pythonu:
1. velké objekty typu `DataFrame` jsou vyhodnocovány efektivněji,
2. aritmetické a logické výrazy projdou naráz základním enginem (pomocí `numexpr`).

**Pozor!** Dobrá obecná pomůcka říká, že není nutné využívat funkci `eval` pro tabulky, které jsou kratší než 10 000 řádků. 

### Použití EVAL

---



Funkce `eval` slouží k efektivnímu vyhodnocení **aritmetických výrazů** na:
* objekty `DataFrame`,
* objekty `Series`.

Její největší výhodou je **rychlost a efektivita**, která spočívá v minimalizaci paměťové zátěže a zrychlení operací.

Když pracuješ **s velkými datovými sadami**, paměťová zátěž a rychlost se stávají klíčovými faktory.

Funkce `eval` optimalizuje výpočetní proces tím, že snižuje množství alokované paměti a zrychluje výpočetní operace.

Ukázka použití funkce `eval`:

### Ukázka EVAL

---

Pro zadaný *dummy dataset* vytvoř **nový sloupeček** `sloupec_D`, podle vzoru:
`A + B * C`

In [1]:
from pandas import DataFrame, eval

In [2]:
data = {'A': [1, 2, 3, 4, 5],
        'B': [6, 7, 8, 9, 10],
        'C': [11, 12, 13, 14, 15]}

In [3]:
df_data = DataFrame(data)

In [5]:
df_data

,A,B,C
0,1,6,11
1,2,7,12
2,3,8,13
3,4,9,14
4,5,10,15


In [9]:
df_data['sloupec_D'] = df_data['A'] + df_data['B'] * df_data['C']

In [10]:
df_data

,A,B,C,sloupec_D
0,1,6,11,67
1,2,7,12,86
2,3,8,13,107
3,4,9,14,130
4,5,10,15,155


<br>

*Alternativní způsob*, jak provést stejný výpočet, pokud máš dlouhou tabulku nebo sloupeček, je použití funkce `eval`:

In [11]:
df_data['sloupecek_D'] = df_data.eval('A + B * C')

<br>

Funkce `eval` zpracovává zadaný `str` podobně jako built-in funkce `eval`.

In [12]:
df_data

,A,B,C,sloupec_D,sloupecek_D
0,1,6,11,67,67
1,2,7,12,86,86
2,3,8,13,107,107
3,4,9,14,130,130
4,5,10,15,155,155


Souhrnně lze říci:
* funkci `eval` použij tehdy, pokud je dataset dlouhý,
* ve všech ostatních scénářích, kde nepotřebuješ šetřit pamět a čas použij klasickou syntaxi.

### Výkon EVAL

---

In [22]:
import numpy as np
import pandas as pd

In [14]:
pocet_radku, pocet_sloupcu = 100_000, 100

In [15]:
rng = np.random.RandomState(42)

In [29]:
dframe_1, dframe_2, dframe_3, dframe_4 = [DataFrame(rng.rand(pocet_radku, pocet_sloupcu)) for _ in range(4)]

In [30]:
%timeit dframe_1 + dframe_2 + dframe_3 + dframe_4

523 ms ± 69.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%timeit pd.eval("dframe_1 + dframe_2 + dframe_3 + dframe_4")

354 ms ± 64.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<br>

### Funkce QUERY

---

Funkce `query` slouží k efektivnímu **filtrování řádků**.

Filtrování probíhá na základě **zadané podmínky**.


Hlavní výhodou jsou:
* **snadná čitelnost**, umožňuje zapisovat podmínky ve snadno čitelné formě, zejména v případě složitějších dotazů,
* **rychlost**, může být rychlejší než tradiční metody filtrování,(zejména pro velké datové sady).

### Ukázka QUERY

---

Pro zadaný *dummy dataset* vyfiltruj takové záznamy, kde:
1. Hodnoty ze sloupce A budou **menší než 4**,
2. hodnoty ze sloupce B budou **větší než 6**,
3. hodnoty ze sloupce C budou **větší než 11**.

In [32]:
data = {'A': [1, 2, 3, 4, 5],
        'B': [6, 7, 8, 9, 10],
        'C': [11, 12, 13, 14, 15]}

In [33]:
df_data = DataFrame(data)

In [34]:
df_data

,A,B,C
0,1,6,11
1,2,7,12
2,3,8,13
3,4,9,14
4,5,10,15


In [39]:
df_filtrovane = df_data[(df_data['A'] < 4) & (df_data['B'] > 6) & (df_data['C'] > 11)]

In [40]:
df_filtrovane

,A,B,C
1,2,7,12
2,3,8,13


Zapisování pomocí *boolean indexování* je ovšem náročné jak pro zápis, tak pro čtení.

<br>

Pomocí funkce `query`:

In [37]:
df_filtrovane = df_data.query('A < 4 and B > 6 and C > 11')

In [38]:
df_filtrovane

,A,B,C
1,2,7,12
2,3,8,13


### Nedostatky QUERY

---

1. **Omezená syntaxe**, má omezenější syntaxi ve srovnání s běžným zápisem v Pythonu a nemusí podporovat všechny operace,
2. **Názvy sloupců**, pokud názvy sloupců obsahují **mezery nebo znaky**, které nejsou platnými identifikátory Pythonu, musíš je uvést v závorkách a použít symbol `@` pro reference na proměnné mimo `DataFrame`.

In [47]:
df_cisla = pd.DataFrame({'Sloupec 1': [1, 2, 3, 4, 5], 'Sloupec 2': [10, 20, 30, 40, 50]})

In [48]:
vystup_1 = df_cisla.query('Sloupec 1 < 3')

SyntaxError: invalid syntax (<unknown>, line 1)

In [49]:
vystup_2 = df_cisla.query('`Sloupec 1` < 3')

In [50]:
vystup_2

,Sloupec 1,Sloupec 2
0,1,10
1,2,20


### Souhrn k EVAL a QUERY

---

Hlavním účelem těchto funkcí je aplikace na skutečně **velké datové sety**.

Rozdíly u drobnějších datasetů **jsou často minimální**.

Velký vliv na výstupný čas dělá parametr `engine`, který se snaží pracovat s knihovnou `numexpr`.

<br>

**🧠 CVIČENÍ 🧠, procvič si funkce EVAL a QUERY**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Vyber studenty s věkem **mezi 20 a 22 lety (včetně)**,
2. vypočtěte *BMI* (Body Mass Index) pro každého studenta a přidejte ho do nového sloupce (vzorec: *váha[kg] / (výška[m] * výška[m])*)
3. vyberte studenty s BMI vyšším než 22.

In [88]:
uzivatele = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Age': [19, 20, 21, 22, 23],
    'Height_cm': [160, 170, 175, 180, 165],
    'Weight_kg': [50, 65, 70, 80, 55],
    'Email': ["alice@gmail.com", "bob@tech-users.com", "charlie91@mail.com", "david_bB@gmail.com", "eevee@my_domain.net"],
    'Address': [
        "Vrbová 215/14, 602 00, Brno, Česká republika",
        "Jilmová 862, 400 11, Ústí nad Labem, Česká republika",
        "Kamenická 37/2, 170 00, Praha, Česká republika",
        "Podlipná 1284/6, 460 01, Liberec, Česká republika",
        "Havlíčkova 1703, 500 02, Hradec Králové, Česká republika"
    ]
}

In [89]:
df_uzivatele = DataFrame(uzivatele)

In [90]:
df_uzivatele

,Name,Age,Height_cm,Weight_kg,Email,Address
0,Alice,19,160,50,alice@gmail.com,"Vrbová 215/14, 602 00, Brno, Česká republika"
1,Bob,20,170,65,bob@tech-users.com,"Jilmová 862, 400 11, Ústí nad Labem, Česká rep..."
2,Charlie,21,175,70,charlie91@mail.com,"Kamenická 37/2, 170 00, Praha, Česká republika"
3,David,22,180,80,david_bB@gmail.com,"Podlipná 1284/6, 460 01, Liberec, Česká republika"
4,Eve,23,165,55,eevee@my_domain.net,"Havlíčkova 1703, 500 02, Hradec Králové, Česká..."


In [68]:
vyber_uzivatel = df_uzivatele.query("20 <= Age <= 22 and Email.str.contains('gmail.com')")

In [69]:
vyber_uzivatel

,Name,Age,Height_m,Weight_kg,Email
3,David,22,180,80,david_bB@gmail.com


In [96]:
df_uzivatele['Height_m'] = df_uzivatele.eval('Height_cm / 100')

In [97]:
df_uzivatele['BMI'] = df_uzivatele.eval('Weight_kg / (Height_m * Height_m)').round(1)

In [98]:
df_uzivatele

,Name,Age,Height_cm,Weight_kg,Email,Address,City,Height_m,BMI
0,Alice,19,160,50,alice@gmail.com,"Vrbová 215/14, 602 00, Brno, Česká republika",Brno,1.60,19.5
1,Bob,20,170,65,bob@tech-users.com,"Jilmová 862, 400 11, Ústí nad Labem, Česká rep...",Ústí nad Labem,1.70,22.5
2,Charlie,21,175,70,charlie91@mail.com,"Kamenická 37/2, 170 00, Praha, Česká republika",Praha,1.75,22.9
3,David,22,180,80,david_bB@gmail.com,"Podlipná 1284/6, 460 01, Liberec, Česká republika",Liberec,1.80,24.7
4,Eve,23,165,55,eevee@my_domain.net,"Havlíčkova 1703, 500 02, Hradec Králové, Česká...",Hradec Králové,1.65,20.2


In [91]:
df_uzivatele["City"] = df_uzivatele["Address"].apply(lambda x: x.split(",")[2])

In [92]:
df_uzivatele

,Name,Age,Height_cm,Weight_kg,Email,Address,City
0,Alice,19,160,50,alice@gmail.com,"Vrbová 215/14, 602 00, Brno, Česká republika",Brno
1,Bob,20,170,65,bob@tech-users.com,"Jilmová 862, 400 11, Ústí nad Labem, Česká rep...",Ústí nad Labem
2,Charlie,21,175,70,charlie91@mail.com,"Kamenická 37/2, 170 00, Praha, Česká republika",Praha
3,David,22,180,80,david_bB@gmail.com,"Podlipná 1284/6, 460 01, Liberec, Česká republika",Liberec
4,Eve,23,165,55,eevee@my_domain.net,"Havlíčkova 1703, 500 02, Hradec Králové, Česká...",Hradec Králové


In [99]:
students_with_high_bmi = df_uzivatele.query('BMI > 22 and City in ["Brno", "Praha"]')

In [100]:
students_with_high_bmi

,Name,Age,Height_cm,Weight_kg,Email,Address,City,Height_m,BMI


<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    monthly_sales = df.resample('M').sum()
    print("Celkový prodej za každý měsíc:")
    print(monthly_sales)

    df_prodeje['weekday'] = df_prodeje.index.weekday
    average_weekday_sales = df_prodeje.groupby('weekday')['prodeje'].mean()
    print("\nPrůměrný prodej za každý den v týdnu:")
    print(average_weekday_sales)

    last_week_sales = df.loc['2022-12-25':, 'sales']
    print("\nDenní prodej za poslední týden:")
    print(last_week_sales)
    ```
</details>

<br>

## Chybějící hodnoty

---

https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.JJu86kKRph1LHt5M7agODQHaHa%26pid%3DApi&f=1&ipt=5cfb9e9f7b451b3a4f7a3ebdc4eb99177d477f20f8145cd818cc4966143f8b2b&ipo=images

Čištění dat je proces, který patří k samotné práci s daty.

Spolu s některými souvisejícími úkony, patří mezi ty více frustrující.

Nesmyslné datové typy, zkomolené časové údaje, nefungující transformace.

In [116]:
df_uzivatele = DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Age': [None, 20, None, 22, 23],
    'Height_cm': [160, 170, 175, None, 165],
    'Weight_kg': [50, 65, 70, 80, 55]
})

In [117]:
df_uzivatele.head(3)

,Name,Age,Height_cm,Weight_kg
0,Alice,NaN,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,NaN,175.0,70


Ihned po načtení je nejlepší data prozkoumat.

Nejenom datové typy ale také mít povědomí **o chybějících hodnotách**:

In [107]:
df_uzivatele.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       5 non-null      object 
 1   Age        3 non-null      float64
 2   Height_cm  4 non-null      float64
 3   Weight_kg  5 non-null      int64  
 4   Email      4 non-null      object 
dtypes: float64(2), int64(1), object(2)
memory usage: 328.0+ bytes


<br>

Pomocí metody `info` si uděláš aspoň povrchní přehled o hodnotách, datových typech a chybějících hodnotách.

<br>

### Kolik mi chybí údajů

---

Vždycky přistupuj k datům s mírnou skepsí.

Otázkou potom není, *JESTLI* mi chybí data, ale *KOLIK* dat mi chybí **a kde**.

In [108]:
chybejici_hodnoty = df_uzivatele.isnull()

In [109]:
chybejici_hodnoty

,Name,Age,Height_cm,Weight_kg,Email
0,False,True,False,False,False
1,False,False,False,False,False
2,False,True,False,False,False
3,False,False,True,False,False
4,False,False,False,False,True


<br>

Pomocí metody `isnull` nahradíš všechny chybějící hodnoty v tabulce/sloupcích pomocí `True`.

Pokud nechybí, nahradí s `False`.

In [110]:
type(chybejici_hodnoty)

pandas.core.frame.DataFrame

Takový výstup ale není zcela reprezentativní.

Proto je potřeba, sečíst všechny `True` hodnoty po sloupcích:

In [112]:
chybejici_hodnoty = df_uzivatele.isnull().sum()

In [113]:
chybejici_hodnoty

Name         0
Age          2
Height_cm    1
Weight_kg    0
Email        1
dtype: int64

<br>

Opět v jednotkách to nemusí zcela prozrazovat chybovost.

Nejlepším indikátorem ale bývají procenta z celkového množství hodnot ve sloupci:

In [122]:
celkem_zaznamu = df_uzivatele.shape[0]  # 0 ... Indexy, 1 ... sloupce

In [123]:
celkem_chybi = chybejici_hodnoty

In [124]:
v_procentech = round(celkem_chybi / celkem_zaznamu * 100, 1)

In [125]:
v_procentech

Name          0.0
Age          40.0
Height_cm    20.0
Weight_kg     0.0
Email        20.0
dtype: float64

<br>

případně celkové množství záznamů ve všech sloupcích:

In [126]:
vsechny_bunky = df_uzivatele.shape[0] * df_uzivatele.shape[1]  # 5 * 4

In [127]:
vsechny_chybejici_hodnoty = chybejici_hodnoty.sum()            # 4

In [128]:
celkem_v_procentech = round(vsechny_chybejici_hodnoty / vsechny_bunky * 100, 1)

In [129]:
celkem_v_procentech

20.0

<br>

V této ukázce tedy chybí 20 % dat a to není málo!

<br>

### Co s chybějícími hodnotami

---

Je důležité vědět, proč máš takové množství chybějících hodnot.

Chybí ti data, protože neexistují, nebo protože nejsou součástí záznamů v datasetu.

V takových krocích je obvykle nutné, pochopit podstatu údajů (dokumentace, specifikace,..).

Nejjednodušší způsoby, jak s chybějícími daty naložit (ale ne efektivní!!):
1. Zahodit chybějící hodnoty,
2. doplnit chybějící hodnoty.

#### Zahodit chybějící hodnoty

In [130]:
df_uzivatele.dropna()

,Name,Age,Height_cm,Weight_kg
1,Bob,20.0,170.0,65
4,Eve,23.0,165.0,55


<br>

V tento okamžik si zahodíš všechny záznamy, které **postrádaly nějakou hodnotu**.

Někdy je jednodušší zahodit sloupeček, který má chybějící hodnotu:

In [131]:
df_uzivatele.dropna(axis=1)

,Name,Weight_kg
0,Alice,50
1,Bob,65
2,Charlie,70
3,David,80
4,Eve,55


#### Doplnit automaticky hodnoty

In [ ]:
df_uzivatele.fillna(0)

<br>

Pomocí funkce `fillna` můžeš vyplnit místo chybějících hodnot předdefinovanou vlastní hodnotu.

Nebo pomocí argumentu pro `method` pracovat s elegantnějším zadáním.

Nahradit chybějící hodnoty hodnotou, která následuje bezprostředně za ní ve stejném sloupci (To má velký smysl u souborů dat, kde mají pozorování nějaké logické pořadí.)

In [144]:
df_uzivatele.fillna(method='bfill', axis=0).fillna(0)

,Name,Age,Height_cm,Weight_kg
0,Alice,20.0,160.0,50
1,Bob,20.0,170.0,65
2,Charlie,22.0,175.0,70
3,David,22.0,165.0,80
4,Eve,23.0,165.0,55
